In [2]:

%run ../../load_magic/storage.py
%run ../../load_magic/dataframes.py
import urllib.request
import re
from bs4 import BeautifulSoup
%pprint
%who

DATA_FOLDER: ../data/
SAVES_FOLDER: ../saves/
Pretty printing has been turned OFF
BeautifulSoup	 DATA_CSV_FOLDER	 DATA_FOLDER	 ENCODING_TYPE	 SAVES_CSV_FOLDER	 SAVES_FOLDER	 SAVES_PICKLE_FOLDER	 attempt_to_pickle	 example_iterrows	 
get_column_descriptions	 load_csv	 load_dataframes	 load_object	 math	 os	 pd	 pickle	 re	 
save_dataframes	 store_objects	 sys	 urllib	 


In [3]:

index_name = 'Building Name'
dataframes_dict = load_dataframes(building_availability_df='building_availability_df')
building_availability_df = dataframes_dict['building_availability_df']
if 'Unnamed: 0' in building_availability_df.columns:
    building_availability_df = building_availability_df.set_index('Unnamed: 0')
elif index_name in building_availability_df.columns:
    building_availability_df = building_availability_df.set_index(index_name)
building_availability_df.index.name = index_name

Attempting to load C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\pickle\building_availability_df.pickle.
No pickle exists - attempting to load C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\csv\building_availability_df.csv.


In [4]:

aside_columns_dict = load_object('aside_columns_dict')
aside_columns_dict

{'A': 'Age', 'HP': 'Hit points', 'Ga': 'Garrison capacity', 'Ar': 'Armor', 'PA': 'Pierce armor', 'LOS': 'Line of Sight', 'Ra': 'Range', 'At': 'Attack', 'W': 'Wood cost', 'S': 'Stone cost', 'G': 'Gold cost', 'CT': 'Construction time in seconds', 'Sz': 'Size in tiles', 'Type': 'Type', 'Intro': 'Introduced in', 'Available': 'Civilization', 'Age': 'Age', 'Time': 'Construction time', 'Use': 'Use', 'Wood': 'Wood', 'Stone': 'Stone', 'Size': 'Size', 'Garrison': 'Garrison', 'Attack': 'Attack', 'BAttack': 'Attack bonuses', 'ROF': 'Rate of Fire', 'Range': 'Range', 'Accuracy': 'Accuracy', 'PSpeed': 'Projectile speed', 'Armor': 'Melee armor', 'PArmor': 'Pierce armor', 'Class': 'Armor classes', 'Required': 'Required research', 'Next': 'Required for', 'MinRange': 'Minimum range', 'UpTo': 'Upgrades to', 'UpCost': 'Upgrade cost', 'UpTime': 'Upgrade time', 'UpFrom': 'Upgraded from', 'Gold': 'Gold', 'Research': 'Required research', 'Civilization': 'Civilization', 'Building': 'Trained at', 'TrainTime': 'T

In [5]:

mask_series = pd.Series(cn in aside_columns_dict for cn in building_availability_df.columns)
building_availability_df.columns[mask_series]

Index(['Age', 'Type', 'Use', 'Stone', 'Wood', 'Gold', 'Range', 'Attack',
       'Size', 'Accuracy', 'Garrison'],
      dtype='object')

In [26]:

cl = building_availability_df.columns
acd = aside_columns_dict.copy()
building_availability_df.columns = [cn if (cn not in acd) else acd[cn] for cn in cl]

In [14]:

for cn in ['Range', 'ROF']:
    building_availability_df[cn] = building_availability_df[cn].fillna(value=0)

In [8]:

building_availability_df['Class'] = building_availability_df['Class'].fillna(value='')
armor_regex = re.compile(r'Building \(\+(\d+)\)')

def f(x):
    match_obj = armor_regex.search(x)
    if match_obj:
        armor = match_obj.group(1)
    else:
        armor = '0'
    
    return int(armor)

building_availability_df['Building_Armor'] = building_availability_df['Class'].map(f)

In [3]:

def get_mixed(x):
    x = str(x)
    if re.search(r'([+.\d]+\s+[^+.\d]+|[^+.\d]+\s+[+.\d]+)', x):
        
        return True
    
    else:
        
        return False

digit_df = building_availability_df.applymap(get_mixed)
for column_name in digit_df.columns[digit_df.any(axis=0)]:
    print()
    print(column_name)
    print(building_availability_df[column_name].unique().tolist())


Attack
[nan, '120 pierce *', '11 pierce', '7  pierce', '6 pierce', '8  pierce', '5 pierce']

Size
['3x3 tiles', '1x1 tiles', '4x4 tiles', '5x5 tiles', '4x1 tiles', '2x2 tiles']

UpTime
[nan, '75 seconds', '50', '30 seconds']

BAttack
[nan, '+40 vs Ship +1 vs Camel', '+2 vs Spearman', '+9 vs Ship +2 vs Spearman +1 vs Camel', '+9 vs ships', '+10 vs Ship +2 vs Spearman +1 vs Camel', '+5 vs Ship +5 vs Building +1 vs Camel', '+7 vs Ship +2 vs Spearman +1 vs Camel']

Use
['Trains and improves archers', 'Trains and improves infantry', 'Improves infantry , archers , cavalry', 'Attacks enemies', 'Train unique units and siege weapons Research certain technologies Attack enemies', 'Builds and improves ships', 'Provides food', 'Generates resources', 'Delay enemy units', 'Shuts enemy units out Lets friendly units through', 'Provides LOS and attacks enemies within range', 'Build and improve ships Attack enemies', 'Provides population', 'Attacks enemies at sight', 'Drop off wood Research logging tec

In [4]:

print(building_availability_df['Class'].unique().tolist())

['Building Standard building', 'Gunpowder unit Building Standard building Stone defense', 'Building (+8) Standard building Castle', nan, 'Building (+24) Standard building Stone defense Wall and gate', 'Building (+20) Standard building Stone defense Wall and gate', 'Building Standard building Stone defense', 'Building Standard building Wall and gate', 'Building (+16) Standard building Stone defense Wall and gate', 'Building']


In [5]:

class_list = ['Building', 'Standard building', 'Stone defense', 'Wall and gate',
              'Gunpowder unit', 'Castle']
class_regex = re.compile(r'({})'.format('|'.join(class_list)))
def get_Class_column(column_name):
    
    def f(column_value):
        column_value = str(column_value)
        return_value = 0
        for match_obj in class_regex.finditer(column_value):
            class_name = match_obj.group(1)
            if class_name == column_name:
                return_value = 1
        
        return return_value

    new_column_name = 'In_{}_Armor_class'.format(re.sub(' +', '_', column_name))
    building_availability_df[new_column_name] = building_availability_df['Class'].map(f)

for resource in class_list:
    get_Class_column(resource)

In [ ]:

building_availability_df['UpCost'] = building_availability_df['UpCost'].fillna(value='')
up_regex = re.compile(r'(\d+) +food *, +(\d+) +wood')
def f(x):
    match_obj = up_regex.search(x)
    if match_obj:
        food = match_obj.group(1)
    else:
        food = '0'
    
    return int(food)

building_availability_df['UpCost_Food'] = building_availability_df['UpCost'].map(f)

In [ ]:

def f(x):
    match_obj = up_regex.search(x)
    if match_obj:
        wood = match_obj.group(2)
    else:
        wood = '0'
    
    return int(wood)

building_availability_df['UpCost_Wood'] = building_availability_df['UpCost'].map(f)

In [27]:

columns_list = building_availability_df.columns
file_path = os.path.join(DATA_CSV_FOLDER, 'building_availability_df.csv')
print('Saving to {}'.format(os.path.abspath(file_path)))
building_availability_df[columns_list].to_csv(file_path)
save_dataframes(include_index=True,
                building_availability_df=building_availability_df[columns_list])
xlsx_folder = os.path.join(DATA_FOLDER, 'xlsx')
os.makedirs(name=xlsx_folder, exist_ok=True)
file_path = os.path.join(xlsx_folder, 'building_availability_df.xlsx')
print('Saving to {}'.format(os.path.abspath(file_path)))
building_availability_df[columns_list].to_excel(file_path)

Saving to C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\data\csv\building_availability_df.csv
Saving to C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\saves\csv\building_availability_df.csv
Saving to C:\Users\dev\Documents\repositories\notebooks\age-of-empires-ii\data\xlsx\building_availability_df.xlsx
